# How much do drivers pay for tolls across the country?

In [1]:
import pandas as pd
import numpy as np

# Data source

FHWA - https://www.fhwa.dot.gov/policyinformation/tollpage/

In [8]:
# Get a requested spreadsheet
# Note that the interstate, non-interstate, and bandt
# spreadsheets in the data folder are modified via 
# spreadsheet application to truncate some 
# non-data lines before and after the content

def get_df(which="non-interstate"):
    
    ret = pd.read_csv("data/" + which + ".csv", header=None)
    cols = []
    
    row_0 = ret.iloc[0].fillna("")
    row_1 = ret.iloc[1].fillna("")
    row_2 = ret.iloc[2].fillna("")
    
    for i in ret.columns:
        c = ""
        if row_0[i] is not None:
            c += str(row_0[i]) + " "
        if row_1[i] is not None:
            c += str(row_1[i]) + " "
        if row_2[i] is not None:
            c += str(row_2[i]) + " "
        
        cols.append(c.strip())
        
    # print cols
    
    ret = ret.drop(0).drop(1).drop(2)#.reset_index()
    
    ret.columns = cols
    
    # cut out out blank-named cols
    keepers = filter(lambda x: x.strip() != "", cols)
    
    ret = ret[keepers]

    
    def money_num(n):
        if type(n) != str:
            return
        return n.replace("$","")
    
    def money_col(c):
        ret["num_" + c] = pd.to_numeric(ret[c].apply(money_num), errors="coerce")

    for mc in ["Maximum Passenger Vehicle Fee","Minimum Passenger Vehicle Fee",
    "Average Passenger Vehicle Cost per Vehicle-Mile",
    "Maximum Truck Fee",
    "Minimum Truck Fee",
    "Average Truck Cost per Vehicle-Mile"]:
        money_col(mc)
    
    ret["State"] = ret["State"].str.replace(" 3/","") # Would be better to use regex
    ret["State"] = ret["State"].str.replace(" 5/","") # but... working fast
    ret["State"] = ret["State"].str.replace(" 7/","")

    ret["State"] = ret["State"].str.strip()
    ret["Length 1/ Miles"] = pd.to_numeric(ret["Length 1/ Miles"],errors="coerce")
    
    
    ret["spreadsheet"] = which
    return ret

get_df(which="interstate").head()

,State,Name of Facility,HPMS Toll ID,Financing or Operating Authority,From,To,Length 1/ Miles,Kilometers,Rural/Urban,Interstate Route,...,Maximum Truck Fee,Minimum Truck Fee,Average Truck Cost per Vehicle-Mile,num_Maximum Passenger Vehicle Fee,num_Minimum Passenger Vehicle Fee,num_Average Passenger Vehicle Cost per Vehicle-Mile,num_Maximum Truck Fee,num_Minimum Truck Fee,num_Average Truck Cost per Vehicle-Mile,spreadsheet
3,California,I-15 Value Pricing Project,16,San Diego Assoc of Gov; CA Dept of Trans,SR 56/ Ted Williams Pkwy,SR 52,8.0,12.87,Urban,15,...,NaN,NaN,NaN,8.0,0.50,NaN,NaN,NaN,NaN,interstate
4,California,I-680 SMART Carpool Lanes,297,Alameda County Congestion Management Agency,SR 84/ Pleasanton,SR 237 Milpitas,14.0,22.53,Urban,680,...,$0.00,$0.00,NaN,8.0,0.30,NaN,0.0,0.0,NaN,interstate
5,California,I-880 / SR 237 Express Connector,298,Santa Clara Valley Transportation Authority,I-880,SR 237,4.0,6.44,Urban,880,...,NaN,NaN,NaN,NaN,0.30,NaN,NaN,NaN,NaN,interstate
6,California,I-110 Express Lanes,313,Los Angeles County Metropolitan Transportation...,Harbor Gateway Transit Center,Adams Blvd,11.0,17.70,Urban,110,...,NaN,NaN,NaN,15.4,0.25,NaN,NaN,NaN,NaN,interstate
7,California,I-10 Express Lanes,338,Los Angeles County Metropolitan Transportation...,Alameda St/Union Station,I-605,14.0,22.53,Urban,10,...,NaN,NaN,NaN,19.6,0.25,NaN,NaN,NaN,NaN,interstate


In [9]:
# get frame for each of the three types of tolls

interstate = get_df(which="interstate")
print "interstate", len(interstate)
non_interstate = get_df(which="non-interstate")
print "non interstate",len(non_interstate)
bandt = get_df("bandt")
print "bandt", len(bandt)

interstate 106
non interstate 138
bandt 112


In [10]:
# combine all into one big "all_df"
frames = [bandt,interstate,non_interstate]
all_df = pd.concat(frames)

In [11]:
# combine just the highway frames 
highway_df = pd.concat([interstate,non_interstate])

### Look at the average price per mile

In [12]:
highway_df["num_Average Passenger Vehicle Cost per Vehicle-Mile"].describe()

count    164.000000
mean       0.149756
std        0.227294
min        0.020000
25%        0.050000
50%        0.100000
75%        0.170000
max        2.200000
Name: num_Average Passenger Vehicle Cost per Vehicle-Mile, dtype: float64

In [13]:
interstate["num_Average Passenger Vehicle Cost per Vehicle-Mile"].describe()

count    79.000000
mean      0.100253
std       0.089743
min       0.020000
25%       0.050000
50%       0.060000
75%       0.130000
max       0.510000
Name: num_Average Passenger Vehicle Cost per Vehicle-Mile, dtype: float64

In [14]:
non_interstate["num_Average Passenger Vehicle Cost per Vehicle-Mile"].describe()

count    85.000000
mean      0.195765
std       0.297195
min       0.030000
25%       0.080000
50%       0.130000
75%       0.190000
max       2.200000
Name: num_Average Passenger Vehicle Cost per Vehicle-Mile, dtype: float64

In [15]:
highway_df["num_Maximum Passenger Vehicle Fee"].describe()

count    206.000000
mean       5.478689
std        7.005890
min        0.350000
25%        1.420000
50%        3.000000
75%        6.995000
max       40.000000
Name: num_Maximum Passenger Vehicle Fee, dtype: float64

In [16]:
highway_df["num_Minimum Passenger Vehicle Fee"].describe()

count    208.000000
mean       1.968942
std        3.307094
min        0.000000
25%        0.500000
50%        0.790000
75%        1.962500
max       28.000000
Name: num_Minimum Passenger Vehicle Fee, dtype: float64

In [31]:
highway_df.sort_values(by="num_Maximum Passenger Vehicle Fee",ascending=False).head()

,Average Passenger Vehicle Cost per Vehicle-Mile,Average Truck Cost per Vehicle-Mile,Both Ways,Electronic Toll Collection System? 2/ No,Electronic Toll Collection System? 4/ No,Federal Authority Source,Fee Type,Financing or Operating Authority,From,Functional Classification 2/,...,To,"Toll Collection One-Way (N,S,E,W)",Yes/Kind,num_Average Passenger Vehicle Cost per Vehicle-Mile,num_Average Truck Cost per Vehicle-Mile,num_Maximum Passenger Vehicle Fee,num_Maximum Truck Fee,num_Minimum Passenger Vehicle Fee,num_Minimum Truck Fee,spreadsheet
12,$0.39,$0.39,NaN,NaN,X,None (not a Federal-aid toll facility),Fixed: Rate does not vary by time of day or tr...,City of Colorado Springs operates under a Term...,"Town of Cascade, US 24 west of Colorado Springs",local,...,"Top of Pikes Peak Mountain -14, 110' elevation",W or UP :-),Booth,0.39,0.39,40.0,40.0,10.00,10.0,non-interstate
48,$0.05,$0.24,X,NaN,NaN,None (not a Federal-aid toll facility),Fixed: Rate does not vary by time of day or tr...,NY State Thruway Authority,Pennsylvania Line,NaN,...,Albany,NaN,E-ZPass,0.05,0.24,33.7,137.9,0.15,0.4,interstate
47,$0.05,$0.24,X,NaN,NaN,None (not a Federal-aid toll facility),Fixed: Rate does not vary by time of day or tr...,NY State Thruway Authority,Pennsylvania Line,NaN,...,Albany,NaN,E-ZPass,0.05,0.24,33.7,137.9,0.15,0.4,interstate
51,$0.05,$0.24,X,NaN,NaN,None (not a Federal-aid toll facility),Fixed: Rate does not vary by time of day or tr...,NY State Thruway Authority,Exit B1 (US 9),NaN,...,Massachusetts Line,NaN,E-ZPass,0.05,0.24,33.7,137.9,0.15,0.4,interstate
49,$0.05,$0.24,X,NaN,NaN,None (not a Federal-aid toll facility),Fixed: Rate does not vary by time of day or tr...,NY State Thruway Authority,New York City,NaN,...,Albany,NaN,E-ZPass,0.05,0.24,33.7,137.9,0.15,0.4,interstate


### Types of fees


In [18]:
highway_df["Fee Type"].value_counts()

Fixed: Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).    156
Fixed Variable: Rate varies by time of day  based on pre-set schedule                                                       38
Dynamic Variable: Rate varies based on current traffic conditions                                                           15
Fixed Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).       1
Name: Fee Type, dtype: int64

In [19]:
interstate["Fee Type"].value_counts()

Fixed: Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).    66
Fixed Variable: Rate varies by time of day  based on pre-set schedule                                                      17
Dynamic Variable: Rate varies based on current traffic conditions                                                          14
Fixed Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).      1
Name: Fee Type, dtype: int64

In [21]:
len(highway_df)

244

In [23]:
len(highway_df[highway_df["Fee Type"].notnull()])

210

In [32]:
# Types of highway toll fees as a percentage

highway_df["Fee Type"].value_counts() / 210


Fixed: Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).    0.742857
Fixed Variable: Rate varies by time of day  based on pre-set schedule                                                      0.180952
Dynamic Variable: Rate varies based on current traffic conditions                                                          0.071429
Fixed Rate does not vary by time of day or traffic conditions (May vary by vehicle/weight class or distance traveled).     0.004762
Name: Fee Type, dtype: float64

### How much data is missing?


In [33]:
len(interstate[interstate["num_Average Passenger Vehicle Cost per Vehicle-Mile"].isnull()])

27

In [34]:
len(interstate[interstate["num_Average Passenger Vehicle Cost per Vehicle-Mile"].notnull()])

79